In [2]:
# Импорт библиотек
import os
import time
import cotel
import GTU
import mat_properties as prop
import numpy as n
import pandas as pd
from scipy.optimize import root

# Основные константы
calctolerance = 10**-3
calcmethod = "hybr"
KPDnasos = 0.8074
KPD = 0.99

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel(
    "streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)


# Состав газов в номинале
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = (
    gas_streams0.at["GTU-PEVD", "N2"],
    gas_streams0.at["GTU-PEVD", "O2"],
    gas_streams0.at["GTU-PEVD", "CO2"],
    gas_streams0.at["GTU-PEVD", "H2O"],
    gas_streams0.at["GTU-PEVD", "Ar"],
)

gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    RP=RP,
)


# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure= gas_streams0.loc["GTU-KU", "P"]
Enthalpies = list(map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures))
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = Enthalpies


####################################################################################################
# Расчет ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, GTU_input, "GTU-KU")
Gas_turbine_res = Gas_turbine.calc()
electric.loc["GTU", "N"] = Gas_turbine_res["N"]
electric.loc["GTU", "KPD"] = Gas_turbine_res["eff"]
electric.loc["DK", "N"] = Gas_turbine_res["Ndk"]
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
gas_streams.loc["GTU-KU", "T"] = Gas_turbine_res["T"]
gas_streams.loc["GTU-KU", "G"] = Gas_turbine_res["G"]
gas_streams.loc["GTU-KU", "P"] = 0.1
Gas_turbine_composition=pd.read_excel("input.xlsx", sheet_name="Gas_composition0", index_col=0)


#Параметры газа на выходе в КУ
gas_streams.loc["GTU-PEVD", "T":"P"] = gas_streams.loc["GTU-KU", "T":"P"]
gas_streams.loc["GTU-PEVD", "G"] = gas_streams.loc["GTU-KU", "G"]
gas_streams.loc["GTU-PEVD", "N2":"Ar"]=Gas_turbine_composition.loc["Fraction", "N2":"Ar"]

# Состав газов при частичной нагрузке
fractiongas = (
    gas_streams.at["GTU-PEVD", "N2"],
    gas_streams.at["GTU-PEVD", "O2"],
    gas_streams.at["GTU-PEVD", "CO2"],
    gas_streams.at["GTU-PEVD", "H2O"],
    gas_streams.at["GTU-PEVD", "Ar"],
)
gas1 = prop.Materials_prop(
    gasmix,
    fractiongas,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    RP=RP,
)

# # Задание энтальпий газа в рабочем режиме
# Temperatures = gas_streams.loc["GTU-KU":"GPK-out", "T"]
# Pressure= gas_streams.loc["GTU-KU", "P"]
# Enthalpies = list(map(lambda x: gas1.p_t(Pressure, x)["h"], Temperatures))
# gas_streams.loc["GTU-KU":"GPK-out", "H"] = Enthalpies

####################################################################################################
# расчет котла целиком
Whole_cotel = cotel.cotel_all(
    KPD,
    KPDnasos,
    gas0,
    gas1,
    water,
    calcmethod,
    gas_streams0,
    water_streams0,
    gas_streams,
    water_streams,
    heaters,
    electric
)
Whole_cotel.calc(calctolerance*100,maxiterations=50)


Whole_cotel = cotel.cotel_all(
    KPD,
    KPDnasos,
    gas0,
    gas1,
    water,
    calcmethod,
    gas_streams0,
    water_streams0,
    gas_streams,
    water_streams,
    heaters,
    electric
)
Whole_cotel.calc(calctolerance,maxiterations=50)


print(
    "Степень сухости пара в ЭВД: ",
    water.p_h(water_streams.at["EVD-IVD", "P"],
              water_streams.at["EVD-IVD", "H"])["Q"],
)
print(
    "Степень сухости пара в ГПК: ",
    water.p_h(water_streams.at["GPK-IND", "P"],
              water_streams.at["GPK-IND", "H"])["Q"],
)

dQ/Q ПЕВД+ИВД+ЭВД -8.791031356481305
dQ/Q ПЕВД+ИВД+ЭВД 4.460253567052171
dQ/Q ПЕВД+ИВД+ЭВД -1.7428699391260398
dQ/Q ПЕВД+ИВД+ЭВД 0.6817926914238391
dQ/Q ПЕВД+ИВД+ЭВД -0.2640149353167553
dQ/Q ПЕВД+ИВД+ЭВД 0.10290190241041719
dQ/Q ПЕВД+ИВД+ЭВД -0.044184386472966054
dQ/Q ППНД+ИНД+ГПК -10.791108338519805
dQ/Q ППНД+ИНД+ГПК 1.4438068988654094
dQ/Q ППНД+ИНД+ГПК -0.296458997134585
dQ/Q ППНД+ИНД+ГПК 0.08223436554563487
dQ/Qsumm -1.907998413294484
dQ/Q ПЕВД+ИВД+ЭВД -1.004418643638858
dQ/Q ПЕВД+ИВД+ЭВД 0.5943046708369627
dQ/Q ПЕВД+ИВД+ЭВД -0.2426221131955529
dQ/Q ПЕВД+ИВД+ЭВД 0.10615291481078139
dQ/Q ПЕВД+ИВД+ЭВД -0.048743161368766774
dQ/Q ППНД+ИНД+ГПК 0.12208253813021987
dQ/Q ППНД+ИНД+ГПК -0.03910727691414827
dQ/Qsumm -0.050615006285228435
Fin:--- 30.43 сек. ---
dQ/Qsumm -0.050615006285228435
dQ/Qvd -0.048743161368766774
dQ/Qnd -0.03910727691414827
dQ/Q ПЕВД+ИВД+ЭВД 2.833883070817306
dQ/Q ПЕВД+ИВД+ЭВД -1.6329337256502423
dQ/Q ПЕВД+ИВД+ЭВД 0.8008003620273297
dQ/Q ПЕВД+ИВД+ЭВД -0.3936525014455797


In [3]:
# Вывод газовых потоков
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,542.010558,0.1,958.869199,503.497387,0.7803,0.1237,0.0301,0.0594,0.0065
GTU-PEVD,542.010558,0.1,958.533980,503.497387,0.7803,0.1237,0.0301,0.0594,0.0065
PEVD-IVD,466.093110,0.1,871.815514,503.497387,0.7803,0.1237,0.0301,0.0594,0.0065
IVD-EVD,287.202751,0.1,673.473449,503.497387,0.7803,0.1237,0.0301,0.0594,0.0065
EVD-PPND,217.643405,0.1,598.508895,503.497387,0.7803,0.1237,0.0301,0.0594,0.0065
PPND-IND,214.987267,0.1,595.667589,503.497387,0.7803,0.1237,0.0301,0.0594,0.0065
IND-GPK,159.570023,0.1,536.704568,503.497387,0.7803,0.1237,0.0301,0.0594,0.0065
GPK-out,96.452388,0.1,470.184116,503.497387,0.7803,0.1237,0.0301,0.0594,0.0065


In [4]:
# Вывод водяных потоков
water_streams

,T,P,H,G,S,X
AIR,NaN,NaN,NaN,NaN,NaN,NaN
PEVD-DROSVD,515.570233,5.747000,3462.819126,63.476967,NaN,NaN
IVD-PEVD,278.208402,6.244957,2781.859249,63.476967,NaN,NaN
EVD-IVD,277.602604,6.244957,1224.351312,63.476967,NaN,NaN
PEN-EVD,149.981492,6.244957,635.679359,63.476967,NaN,NaN
BND-PEN,149.005609,0.463607,627.890050,63.476967,NaN,NaN
PPND-DROSND,196.142233,0.457000,2849.837529,13.473814,NaN,NaN
IND-PPND,149.005609,0.463607,2744.726042,13.473814,NaN,NaN
GPK-IND,146.379789,0.463607,616.596225,76.950781,NaN,NaN
GPK-REC,146.379789,0.463607,616.596225,90.832910,NaN,NaN


In [ ]:
water_streams0['PEVD-DROSVD':'REC-GPK']

In [ ]:
Errors_water= abs(water_streams.loc['PEVD-DROSVD':'REC-GPK','T':'G'] -water_streams0.loc['PEVD-DROSVD':'REC-GPK','T':'G'])/water_streams0.loc['PEVD-DROSVD':'REC-GPK','T':'G']*100
Errors_water

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,10))
Qg =[0,
    heaters.loc['PEVD','Qg'],
    heaters.loc['PEVD','Qg']+heaters.loc['IVD','Qg'],
    heaters.loc['PEVD','Qg']+heaters.loc['IVD','Qg']+heaters.loc['EVD','Qg'],
    heaters.loc['PEVD','Qg']+heaters.loc['IVD','Qg']+heaters.loc['EVD','Qg']+heaters.loc['PPND','Qg'],
    heaters.loc['PEVD','Qg']+heaters.loc['IVD','Qg']+heaters.loc['EVD','Qg']+heaters.loc['PPND','Qg']+heaters.loc['IND','Qg'],
    heaters.loc['PEVD','Qg']+heaters.loc['IVD','Qg']+heaters.loc['EVD','Qg']+heaters.loc['PPND','Qg']+heaters.loc['IND','Qg']+heaters.loc['GPK','Qg']
    ]
Qw =[0,
    heaters.loc['PEVD','Qg'],
    heaters.loc['PEVD','Qg']+heaters.loc['IVD','Qg'],
    heaters.loc['PEVD','Qg']+heaters.loc['IVD','Qg'],
    heaters.loc['PEVD','Qg']+heaters.loc['IVD','Qg']+heaters.loc['EVD','Qg'],
    heaters.loc['PEVD','Qg']+heaters.loc['IVD','Qg']+heaters.loc['EVD','Qg'],
    heaters.loc['PEVD','Qg']+heaters.loc['IVD','Qg']+heaters.loc['EVD','Qg']+heaters.loc['PPND','Qg'],
    heaters.loc['PEVD','Qg']+heaters.loc['IVD','Qg']+heaters.loc['EVD','Qg']+heaters.loc['PPND','Qg']+heaters.loc['IND','Qg'],
    heaters.loc['PEVD','Qg']+heaters.loc['IVD','Qg']+heaters.loc['EVD','Qg']+heaters.loc['PPND','Qg']+heaters.loc['IND','Qg'],
    heaters.loc['PEVD','Qg']+heaters.loc['IVD','Qg']+heaters.loc['EVD','Qg']+heaters.loc['PPND','Qg']+heaters.loc['IND','Qg']+heaters.loc['GPK','Qg']
    ]
Tg = gas_streams.loc['GTU-PEVD':'GPK-out','T']
Tw = [water_streams.loc['PEVD-DROSVD','T'],
      water_streams.loc['IVD-PEVD','T'],
      water_streams.loc['IVD-PEVD','T'],
      water_streams.loc['EVD-IVD','T'],
      water_streams.loc['PEN-EVD','T'],
      water_streams.loc['PPND-DROSND','T'],
      water_streams.loc['IND-PPND','T'],
      water_streams.loc['IND-PPND','T'],
      water_streams.loc['GPK-REC','T'],
      water_streams.loc['REC-GPK','T'],
     ]
plt.plot(Qg,Tg, color = 'red')
plt.plot(Qw[0:2],Tw[0:2], color = 'blue')
plt.plot(Qw[1:3],Tw[1:3], color = 'blue')
plt.plot(Qw[3:5],Tw[3:5], color = 'blue')
plt.plot(Qw[5:7],Tw[5:7], color = 'blue')
plt.plot(Qw[6:8],Tw[6:8], color = 'blue')
plt.plot(Qw[8:10],Tw[8:10], color = 'blue')
plt.xlabel('Q')
plt.ylabel('T')
plt.legend(['gas','water'])
plt.show()